In [65]:
# 1️⃣ Hospital & department capacity setup (UKE-focused)

import random

# --- ICU units (real counts) -----------------------------------------------
# Each ICU must reserve 1 bed for resuscitation => operational_beds = total_beds - 1
ICU_TOTAL_BEDS = {
    "neurochirurgical_icu": 12,
    "neurological_icu": 12,
    "interdis_stage1": 12,     # UKE 1C
    "interdis_stage2": 12,     # UKE 1D
    "interdis_stage3": 12,     # UKE 1E
    "surgical_icu": 12,        # UKE 1F
    "internal_medicine_icu": 12,  # UKE 1G
    "cardio_icu": 12,          # UKE H1b (cardiology)
    "cardio_surgery_icu": 12,  # UKE H1b (cardiac surgery)
    "vascular_cardiac_icu": 8  # UKE H2b
}

ICU_OPERATIONAL_BEDS = {
    k: max(v - 1, 0) for k, v in ICU_TOTAL_BEDS.items()
}

# --- Non-ICU departments (simulated sizes) ----------------------------------
# Rule of thumb: Large=50, Medium=30, Small=20
DEPARTMENTS_SIZE = {
    # Large
    "internal_medicine": 50,
    "neurology": 50,
    "surgery_general": 50,

    # Medium
    "cardiology": 30,
    "pulmonology": 30,
    "gastroenterology": 30,
    "orthopedics": 30,
    "surgery_trauma": 30,
    "vascular_surgery": 30,

    # Small
    "oncology": 20,
    "pediatrics": 20,
    "obstetrics_gynecology": 20,
    "psychiatry": 20,
    "palliative": 20,
    "imc_internal_medicine": 20,
}

# Combine into a single capacity registry (operational bed counts)
OPER_BEDS = {
    **ICU_OPERATIONAL_BEDS,
    **DEPARTMENTS_SIZE
}

# --- Availability sampling ---------------------------------------------------
# Availability is sampled per patient/event: 0%..50% free, with a small chance of 0% hard block
def sample_availability_fraction(hard_block_prob: float = 0.10) -> float:
    """Return a fraction of free capacity in [0.0, 0.5]. Sometimes forced to 0.0."""
    if random.random() < hard_block_prob:
        return 0.0
    return random.uniform(0.0, 0.5)

def capacity_snapshot_normalized(hard_block_prob: float = 0.10) -> dict:
    """
    Produce a dict of normalized capacity scores per department (0..1),
    computed as available_beds / operational_beds for the current moment.
    """
    snap = {}
    for dept, oper_beds in OPER_BEDS.items():
        if oper_beds <= 0:
            snap[dept] = 0.0
            continue
        frac = sample_availability_fraction(hard_block_prob)
        available = int(round(oper_beds * frac))
        snap[dept] = available / oper_beds
    return snap

# Convenience lists if needed later
ICU_DEPARTMENTS = list(ICU_OPERATIONAL_BEDS.keys())
NON_ICU_DEPARTMENTS = list(DEPARTMENTS_SIZE.keys())
ALL_DEPARTMENTS = list(OPER_BEDS.keys())

# Sanity print (optional)
print("ICU operational beds:", ICU_OPERATIONAL_BEDS)
print("Non-ICU sizes:", DEPARTMENTS_SIZE)


ICU operational beds: {'neurochirurgical_icu': 11, 'neurological_icu': 11, 'interdis_stage1': 11, 'interdis_stage2': 11, 'interdis_stage3': 11, 'surgical_icu': 11, 'internal_medicine_icu': 11, 'cardio_icu': 11, 'cardio_surgery_icu': 11, 'vascular_cardiac_icu': 7}
Non-ICU sizes: {'internal_medicine': 50, 'neurology': 50, 'surgery_general': 50, 'cardiology': 30, 'pulmonology': 30, 'gastroenterology': 30, 'orthopedics': 30, 'surgery_trauma': 30, 'vascular_surgery': 30, 'oncology': 20, 'pediatrics': 20, 'obstetrics_gynecology': 20, 'psychiatry': 20, 'palliative': 20, 'imc_internal_medicine': 20}


In [66]:
# 2️⃣ Patient generator using real ICU and simulated non-ICU capacities

import uuid
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Fixed suspected conditions for now
SUSPECTED_CONDITIONS = [
    "cardiac", "respiratory", "neurological",
    "gastrointestinal", "oncology", "infection", "trauma"
]

HOME_TYPES = ["private", "nursing_home"]

def generate_patient():
    # Static patient attributes
    patient_id = str(uuid.uuid4())[:8]
    age = np.random.randint(18, 95)
    home_type = np.random.choice(HOME_TYPES)
    suspected_condition = np.random.choice(SUSPECTED_CONDITIONS)
    
    # EMS and ED timing
    ems_start_time = datetime(2025, 8, 7) + timedelta(
        hours=np.random.randint(0, 24),
        minutes=np.random.randint(0, 60)
    )
    ems_triage_code = np.random.randint(1, 5)  # 1=most urgent, 4=least
    ems_target_hospital = "UKE"
    ed_arrival_time = ems_start_time + timedelta(minutes=np.random.randint(5, 30))

    # Capacity snapshot for all departments (normalized)
    capacities = capacity_snapshot_normalized()

    # Target: for now, simple suitability score for ward assignment
    ward_assignment_suitability = np.random.uniform(0, 1)

    # Combine all fields
    record = {
        "patient_id": patient_id,
        "age": age,
        "home_type": home_type,
        "suspected_condition": suspected_condition,
        "ems_start_time": ems_start_time,
        "ems_triage_code": ems_triage_code,
        "ems_target_hospital": ems_target_hospital,
        "ed_arrival_time": ed_arrival_time,
    }

    # Add department capacities as features
    for dept, cap in capacities.items():
        record[f"cap_{dept}"] = cap

    # Output target(s)
    record["ward_assignment_suitability"] = ward_assignment_suitability

    return record

def generate_dataset(n_patients=500):
    patients = [generate_patient() for _ in range(n_patients)]
    return pd.DataFrame(patients)

# Quick test
df = generate_dataset(5)
print(df.head())
print("\nShape:", df.shape)


  patient_id  age     home_type suspected_condition      ems_start_time  \
0   6e1bb0ac   19       private             cardiac 2025-08-07 11:43:00   
1   7f0fe7ea   49  nursing_home        neurological 2025-08-07 16:28:00   
2   250eac77   80  nursing_home             cardiac 2025-08-07 01:52:00   
3   6610a015   91  nursing_home              trauma 2025-08-07 00:54:00   
4   864f9fc5   73  nursing_home            oncology 2025-08-07 08:28:00   

   ems_triage_code ems_target_hospital     ed_arrival_time  \
0                3                 UKE 2025-08-07 12:05:00   
1                4                 UKE 2025-08-07 16:55:00   
2                2                 UKE 2025-08-07 02:20:00   
3                2                 UKE 2025-08-07 01:08:00   
4                3                 UKE 2025-08-07 08:45:00   

   cap_neurochirurgical_icu  cap_neurological_icu  ...  cap_orthopedics  \
0                  0.272727              0.090909  ...         0.466667   
1                  0.36363

In [67]:
# Cell “3A” — ICU mapping + label functions
# ICU preference by condition (simple, editable)
ICU_PREFS = {
    "cardiac": ["cap_cardio_icu", "cap_cardio_surgery_icu", "cap_vascular_cardiac_icu"],
    "respiratory": ["cap_internal_medicine_icu", "cap_interdis_stage1", "cap_interdis_stage2", "cap_interdis_stage3"],
    "neurological": ["cap_neurological_icu", "cap_neurochirurgical_icu", "cap_interdis_stage1"],
    "infection": ["cap_internal_medicine_icu", "cap_interdis_stage2"],
    "trauma": ["cap_surgical_icu", "cap_interdis_stage3"],
    "oncology": ["cap_internal_medicine_icu", "cap_interdis_stage2"],
    "gastrointestinal": ["cap_surgical_icu", "cap_internal_medicine_icu"]
}

def best_relevant_icu_capacity(rec: dict) -> float:
    """Return the max normalized capacity across relevant ICUs for this condition."""
    cond = rec["suspected_condition"]
    keys = ICU_PREFS.get(cond, [])
    if not keys:
        return 0.0
    vals = [float(rec.get(k, 0.0)) for k in keys]
    return max(vals) if vals else 0.0

def synth_icu_labels(rec: dict) -> dict:
    """
    Make labels depend on ICU capacity + acuity.
    - icu_assignment_suitability: higher if relevant ICU has capacity.
    - icu_bottleneck_risk: high when relevant ICU is near zero.
    - pathway_latency_min: inflates when bottleneck risk is high (ED boarding).
    - route_plan: EMS→ICU if urgent and capacity; else EMS→ED→ICU.
    """
    cap_rel = best_relevant_icu_capacity(rec)            # 0..1
    triage = int(rec["ems_triage_code"])                 # 1 (most urgent) .. 4
    urgent = (triage <= 2)

    # Suitability: base + positive weight on relevant ICU capacity, small noise
    base_by_cond = {
        "cardiac": 0.70, "respiratory": 0.68, "neurological": 0.67,
        "infection": 0.62, "trauma": 0.66, "oncology": 0.60, "gastrointestinal": 0.63
    }
    base = base_by_cond.get(rec["suspected_condition"], 0.62)
    suitability = base + 0.35*cap_rel - (0.05 if urgent and cap_rel < 0.15 else 0.0)
    suitability = float(np.clip(suitability + np.random.normal(0, 0.03), 0, 1))

    # Bottleneck risk: inverse of capacity with urgency emphasis
    bottleneck = 1.0 - cap_rel
    if urgent:
        bottleneck = np.clip(bottleneck + 0.05, 0, 1)

    # Latency model (minutes): base + penalty for bottleneck
    # Base ~ 60–120 min; add 120–240 min when bottleneck high (boarding)
    base_latency = np.random.uniform(60, 120)
    penalty = np.interp(bottleneck, [0, 1], [0, np.random.uniform(120, 240)])
    pathway_latency_min = float(base_latency + penalty)

    # Route plan decision
    if urgent and cap_rel >= 0.25:
        route_plan = "EMS→ICU"
    else:
        route_plan = "EMS→ED→ICU"

    return {
        "icu_assignment_suitability": suitability,
        "icu_bottleneck_risk": float(np.clip(bottleneck, 0, 1)),
        "pathway_latency_min": pathway_latency_min,
        "route_plan": route_plan
    }


In [68]:
# Cell 3B — Patch generate_patient() to use ICU-driven labels
# --- ICU preferences by suspected condition
ICU_PREFS = {
    "cardiac": ["cap_cardio_icu","cap_cardio_surgery_icu","cap_vascular_cardiac_icu"],
    "respiratory": ["cap_internal_medicine_icu","cap_interdis_stage1","cap_interdis_stage2","cap_interdis_stage3"],
    "neurological": ["cap_neurological_icu","cap_neurochirurgical_icu","cap_interdis_stage1"],
    "infection": ["cap_internal_medicine_icu","cap_interdis_stage2"],
    "trauma": ["cap_surgical_icu","cap_interdis_stage3"],
    "oncology": ["cap_internal_medicine_icu","cap_interdis_stage2"],
    "gastrointestinal": ["cap_surgical_icu","cap_internal_medicine_icu"],
}

def best_relevant_icu_capacity(rec: dict) -> float:
    keys = ICU_PREFS.get(rec["suspected_condition"], [])
    vals = [float(rec.get(k, 0.0)) for k in keys]
    return max(vals) if vals else 0.0

def synth_icu_labels(rec: dict) -> dict:
    cap_rel = best_relevant_icu_capacity(rec)  # 0..1
    triage = int(rec["ems_triage_code"])
    urgent = (triage <= 2)

    # base suitability by condition
    base = {
        "cardiac":0.70,"respiratory":0.68,"neurological":0.67,
        "infection":0.62,"trauma":0.66,"oncology":0.60,"gastrointestinal":0.63
    }.get(rec["suspected_condition"], 0.62)

    suitability = base + 0.35*cap_rel - (0.05 if urgent and cap_rel < 0.15 else 0.0)
    suitability = float(np.clip(suitability + np.random.normal(0,0.03), 0, 1))

    bottleneck = 1.0 - cap_rel
    if urgent:
        bottleneck = np.clip(bottleneck + 0.05, 0, 1)

    base_latency = np.random.uniform(60, 120)  # base in minutes
    penalty = np.interp(bottleneck, [0,1], [0, np.random.uniform(120,240)])
    pathway_latency_min = float(base_latency + penalty)

    route_plan = "EMS→ICU" if (urgent and cap_rel >= 0.25) else "EMS→ED→ICU"

    return {
        "icu_assignment_suitability": suitability,
        "icu_bottleneck_risk": float(bottleneck),
        "pathway_latency_min": pathway_latency_min,
        "route_plan": route_plan
    }

# --- Updated generator with ICU + Non-ICU capacities and ICU labels
def generate_patient():
    patient_id = str(uuid.uuid4())[:8]

    # Demographics
    age = np.random.randint(18, 95)
    home_type = random.choice(['private', 'nursing_home'])

    # Clinical presentation
    suspected_condition = random.choice([
        'cardiac', 'respiratory', 'neurological',
        'infection', 'trauma', 'oncology', 'gastrointestinal'
    ])

    # EMS & ED timing
    ems_start_time = datetime(2025, 8, 7) + timedelta(
        hours=np.random.randint(0, 24),
        minutes=np.random.randint(0, 60)
    )
    ems_triage_code = np.random.choice([1, 2, 3, 4], p=[0.15, 0.35, 0.35, 0.15])
    ems_target_hospital = "UKE"
    ed_arrival_time = ems_start_time + timedelta(minutes=np.random.randint(5, 25))

    day_of_week = ed_arrival_time.strftime("%A")
    hour_of_day = ed_arrival_time.hour

    # ICU capacities (n-1 beds operational)
    icu_capacities = {
        'neurochirurgical_icu': 11,
        'neurological_icu': 11,
        'interdis_stage1': 11,
        'interdis_stage2': 11,
        'interdis_stage3': 11,
        'surgical_icu': 11,
        'internal_medicine_icu': 11,
        'cardio_icu': 11,
        'cardio_surgery_icu': 11,
        'vascular_cardiac_icu': 7
    }
    icu_caps = {f"cap_{k}": round(np.random.randint(0, beds + 1) / beds, 2)
                for k, beds in icu_capacities.items()}

    # Non-ICU capacities (0–50% availability)
    non_icu_sizes = {
        'internal_medicine': 50, 'neurology': 50, 'surgery_general': 50,
        'cardiology': 30, 'pulmonology': 30, 'gastroenterology': 30,
        'orthopedics': 30, 'surgery_trauma': 30, 'vascular_surgery': 30,
        'oncology': 20, 'pediatrics': 20, 'obstetrics_gynecology': 20,
        'psychiatry': 20, 'palliative': 20, 'imc_internal_medicine': 20
    }
    non_icu_caps = {f"cap_{k}": round(np.random.randint(0, int(beds*0.5) + 1) / beds, 2)
                    for k, beds in non_icu_sizes.items()}

    # Assemble record
    record = {
        "patient_id": patient_id,
        "age": age,
        "home_type": home_type,
        "suspected_condition": suspected_condition,
        "ems_start_time": ems_start_time,
        "ems_triage_code": int(ems_triage_code),
        "ems_target_hospital": ems_target_hospital,
        "ed_arrival_time": ed_arrival_time,
        "day_of_week": day_of_week,
        "hour_of_day": hour_of_day,
        **icu_caps,
        **non_icu_caps,
    }

    # ICU-driven labels
    record.update(synth_icu_labels(record))
    return record





In [69]:
# Cell 4 — Regenerate dataset (now includes ICU-driven labels)

# Rebuild a fresh dataset with ICU-driven labels
df = pd.DataFrame(generate_dataset(n_patients=500))

# quick hygiene
assert df.isna().sum().sum() == 0
assert (df["ems_triage_code"].between(1,4)).all()
print(df.shape, "cols:", len(df.columns))
display(df.head(2)[[
    "suspected_condition","ems_triage_code","icu_assignment_suitability",
    "icu_bottleneck_risk","pathway_latency_min","route_plan"
]])


(500, 39) cols: 39


,suspected_condition,ems_triage_code,icu_assignment_suitability,icu_bottleneck_risk,pathway_latency_min,route_plan
0,infection,3,0.645189,0.91,289.542891,EMS→ED→ICU
1,oncology,3,0.716335,0.64,153.635878,EMS→ED→ICU


In [70]:
# Cell 5 — Feature setup

FEATS_CAT = ["home_type", "suspected_condition", "day_of_week"]
FEATS_NUM = [
    "age", "ems_triage_code", "hour_of_day"
] + [c for c in df.columns if c.startswith("cap_")]
TARGETS = ["icu_assignment_suitability", "icu_bottleneck_risk"]

X = df[FEATS_CAT + FEATS_NUM]
y = df[TARGETS]

# Split
X_train, X_tmp, y_train, y_tmp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_tmp, y_tmp, test_size=0.5, random_state=42)

# Preprocessor
pre = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), FEATS_CAT),
    ("num", StandardScaler(), FEATS_NUM)
])

Xtr_np = pre.fit_transform(X_train)
Xva_np = pre.transform(X_val)
Xte_np = pre.transform(X_test)

ytr_np = y_train.values.astype("float32")
yva_np = y_val.values.astype("float32")
yte_np = y_test.values.astype("float32")

print("Shapes:", Xtr_np.shape, Xva_np.shape, Xte_np.shape)

# 📦 2 — Torch datasets
Xtr_t = torch.tensor(Xtr_np, dtype=torch.float32)
ytr_t = torch.tensor(ytr_np, dtype=torch.float32)
Xva_t = torch.tensor(Xva_np, dtype=torch.float32)
yva_t = torch.tensor(yva_np, dtype=torch.float32)
Xte_t = torch.tensor(Xte_np, dtype=torch.float32)
yte_t = torch.tensor(yte_np, dtype=torch.float32)

train_loader = DataLoader(TensorDataset(Xtr_t, ytr_t), batch_size=64, shuffle=True)
val_loader   = DataLoader(TensorDataset(Xva_t, yva_t), batch_size=256)
test_loader  = DataLoader(TensorDataset(Xte_t, yte_t), batch_size=256)


Shapes: (300, 39) (100, 39) (100, 39)


In [71]:
# 📦 3 — Multi-task model
class MLP_MultiTask(nn.Module):
    def __init__(self, d_in, hidden1=128, hidden2=64):
        super().__init__()
        self.shared = nn.Sequential(
            nn.Linear(d_in, hidden1), nn.ReLU(), nn.Dropout(0.1),
            nn.Linear(hidden1, hidden2), nn.ReLU()
        )
        # Heads
        self.head_assign = nn.Linear(hidden2, 1)  # icu_assignment_suitability
        self.head_bottle = nn.Linear(hidden2, 1)  # icu_bottleneck_risk

    def forward(self, x):
        shared_out = self.shared(x)
        out_assign = self.head_assign(shared_out)
        out_bottle = self.head_bottle(shared_out)
        return out_assign, out_bottle

input_dim = Xtr_np.shape[1]
model = MLP_MultiTask(input_dim)


In [72]:
# 📦 4 — Training loop
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

best_val = float("inf")
patience, bad = 8, 0

for epoch in range(1, 51):
    # Train
    model.train(); train_loss = 0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        pred_assign, pred_bottle = model(xb)
        loss1 = criterion(pred_assign, yb[:, [0]])
        loss2 = criterion(pred_bottle, yb[:, [1]])
        loss = loss1 + loss2  # equal weighting
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * xb.size(0)
    train_loss /= len(train_loader.dataset)

    # Val
    model.eval(); val_loss = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            pred_assign, pred_bottle = model(xb)
            loss1 = criterion(pred_assign, yb[:, [0]])
            loss2 = criterion(pred_bottle, yb[:, [1]])
            loss = loss1 + loss2
            val_loss += loss.item() * xb.size(0)
    val_loss /= len(val_loader.dataset)

    print(f"Epoch {epoch:02d} | Train {train_loss:.4f} | Val {val_loss:.4f}")

    if val_loss < best_val - 1e-6:
        best_val = val_loss
        best_state = {k: v.cpu().clone() for k,v in model.state_dict().items()}
        bad = 0
    else:
        bad += 1
        if bad >= patience:
            print("Early stopping")
            break

model.load_state_dict(best_state)


Epoch 01 | Train 0.4792 | Val 0.2642
Epoch 02 | Train 0.1680 | Val 0.1062
Epoch 03 | Train 0.1184 | Val 0.1455
Epoch 04 | Train 0.1178 | Val 0.1075
Epoch 05 | Train 0.0778 | Val 0.0893
Epoch 06 | Train 0.0695 | Val 0.0853
Epoch 07 | Train 0.0659 | Val 0.0756
Epoch 08 | Train 0.0591 | Val 0.0711
Epoch 09 | Train 0.0541 | Val 0.0750
Epoch 10 | Train 0.0505 | Val 0.0736
Epoch 11 | Train 0.0479 | Val 0.0684
Epoch 12 | Train 0.0449 | Val 0.0660
Epoch 13 | Train 0.0445 | Val 0.0639
Epoch 14 | Train 0.0396 | Val 0.0673
Epoch 15 | Train 0.0402 | Val 0.0662
Epoch 16 | Train 0.0379 | Val 0.0646
Epoch 17 | Train 0.0356 | Val 0.0639
Epoch 18 | Train 0.0328 | Val 0.0629
Epoch 19 | Train 0.0301 | Val 0.0634
Epoch 20 | Train 0.0324 | Val 0.0631
Epoch 21 | Train 0.0292 | Val 0.0646
Epoch 22 | Train 0.0290 | Val 0.0628
Epoch 23 | Train 0.0264 | Val 0.0611
Epoch 24 | Train 0.0232 | Val 0.0626
Epoch 25 | Train 0.0233 | Val 0.0627
Epoch 26 | Train 0.0208 | Val 0.0644
Epoch 27 | Train 0.0223 | Val 0.0638
E

<All keys matched successfully>

In [73]:
# 📦 5 — Test evaluation
model.eval()
assign_preds, bottle_preds = [], []
with torch.no_grad():
    for xb, _ in test_loader:
        pa, pb = model(xb)
        assign_preds.append(pa.cpu().numpy())
        bottle_preds.append(pb.cpu().numpy())

assign_preds = np.vstack(assign_preds).ravel()
bottle_preds = np.vstack(bottle_preds).ravel()

mse_assign = mean_squared_error(yte_np[:, 0], assign_preds)
mae_assign = mean_absolute_error(yte_np[:, 0], assign_preds)
mse_bottle = mean_squared_error(yte_np[:, 1], bottle_preds)
mae_bottle = mean_absolute_error(yte_np[:, 1], bottle_preds)

print(f"Assignment suitability → MSE {mse_assign:.4f} | MAE {mae_assign:.4f}")
print(f"Bottleneck risk       → MSE {mse_bottle:.4f} | MAE {mae_bottle:.4f}")


Assignment suitability → MSE 0.0117 | MAE 0.0902
Bottleneck risk       → MSE 0.0565 | MAE 0.1927


In [74]:
# Per-head test metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error
model.eval()
with torch.no_grad():
    pa, pb = [], []
    for xb, _ in test_loader:
        a, b = model(xb)
        pa.append(a.cpu().numpy()); pb.append(b.cpu().numpy())
assign_preds = np.vstack(pa).ravel()
bottle_preds = np.vstack(pb).ravel()

mse_assign = mean_squared_error(yte_np[:,0], assign_preds)
mae_assign = mean_absolute_error(yte_np[:,0], assign_preds)
mse_bottle = mean_squared_error(yte_np[:,1], bottle_preds)
mae_bottle = mean_absolute_error(yte_np[:,1], bottle_preds)
print(f"Assign MSE {mse_assign:.4f} | MAE {mae_assign:.4f}")
print(f"Bottle MSE {mse_bottle:.4f} | MAE {mae_bottle:.4f}")


Assign MSE 0.0117 | MAE 0.0902
Bottle MSE 0.0565 | MAE 0.1927


In [75]:
# Reuse 'pre' and the fitted transformer from earlier. Check that ICU capacity now matters (quick permutation on assignment head)
cat_names = pre.named_transformers_["cat"].get_feature_names_out(FEATS_CAT)
num_names = np.array(FEATS_NUM, dtype=str)
feature_names = np.concatenate([cat_names, num_names])

def predict_assign_numpy(X_np):
    t = torch.tensor(X_np, dtype=torch.float32)
    model.eval()
    with torch.no_grad():
        a, _ = model(t)
    return a.cpu().numpy().ravel()

from sklearn.metrics import mean_squared_error
import numpy as np, pandas as pd

base = mean_squared_error(yte_np[:,0], predict_assign_numpy(Xte_np))
drops = []
Xperm = Xte_np.copy()
rng = np.random.default_rng(42)
for j in range(Xperm.shape[1]):
    col = Xperm[:, j].copy()
    rng.shuffle(Xperm[:, j])
    mse = mean_squared_error(yte_np[:,0], predict_assign_numpy(Xperm))
    drops.append(mse - base)
    Xperm[:, j] = col

imp = pd.DataFrame({"feature": feature_names, "mse_increase": drops}).sort_values("mse_increase", ascending=False)
display(imp.head(15))


,feature,mse_increase
19,cap_surgical_icu,0.001793
2,suspected_condition_cardiac,0.001591
31,cap_surgery_trauma,0.001273
0,home_type_nursing_home,0.001192
18,cap_interdis_stage3,0.001161
15,cap_neurological_icu,0.001051
1,home_type_private,0.000893
21,cap_cardio_icu,0.000785
36,cap_psychiatry,0.000756
38,cap_imc_internal_medicine,0.000711


In [76]:
# --- Combined P1-P3: ICU-aware permutation importance (numeric only) ---
from sklearn.inspection import permutation_importance
import numpy as np

# Use the same numeric test set you trained on
X_test_np = X_test.values.astype(np.float32)  # assumes X_test is already one-hot encoded
y_test_np = y_test.values.astype(np.float32)

# 1. Baseline test MSE
y_pred_np = model(torch.tensor(X_test_np)).detach().numpy().ravel()
baseline_mse = mean_squared_error(y_test_np, y_pred_np)
print(f"Baseline test MSE: {baseline_mse:.4f}")

# 2. Permutation importance
def model_predict(X):
    X_t = torch.tensor(X, dtype=torch.float32)
    return model(X_t).detach().numpy().ravel()

perm_results = permutation_importance(
    estimator=model_predict,
    X=X_test_np,
    y=y_test_np,
    scoring="neg_mean_squared_error",
    n_repeats=10,
    random_state=42
)

# Build DataFrame
perm_df = pd.DataFrame({
    "feature": feature_names,  # from your encoder / preprocessing step
    "mse_increase": -perm_results.importances_mean
}).sort_values(by="mse_increase", ascending=False)

print("\nTop features by permutation importance:")
display(perm_df.head(15))

# 3. Group features
icu_cols = [f for f in feature_names if f.startswith("cap_") and "_icu" in f]
non_icu_cols = [f for f in feature_names if f.startswith("cap_") and "_icu" not in f]
other_cols = [f for f in feature_names if f not in icu_cols + non_icu_cols]

def group_importance(cols, label):
    return {
        "group": label,
        "perm_mse_increase": perm_df.loc[perm_df["feature"].isin(cols), "mse_increase"].mean()
    }

group_df = pd.DataFrame([
    group_importance(icu_cols, "ICU capacity"),
    group_importance(non_icu_cols, "Non-ICU capacity"),
    group_importance(other_cols, "Other")
])

print("\nGroup-level permutation importance:")
display(group_df.sort_values(by="perm_mse_increase", ascending=False))


ValueError: could not convert string to float: 'private'

In [ ]:
# Cell PI-2 — Permutation importance (on test set)
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt

def permutation_importance_torch(predict_fn, X, y, n_repeats=10, seed=42):
    rng = np.random.default_rng(seed)
    baseline = mean_squared_error(y, predict_fn(X))
    importances = np.zeros(X.shape[1], dtype=float)

    for j in range(X.shape[1]):
        losses = []
        X_perm = X.copy()
        for _ in range(n_repeats):
            rng.shuffle(X_perm[:, j])   # in-place column shuffle
            y_hat = predict_fn(X_perm)
            losses.append(mean_squared_error(y, y_hat))
        importances[j] = np.mean(losses) - baseline
    return importances, baseline

imps, base_mse = permutation_importance_torch(mlp_predict_numpy, X_test_np, y_test_np, n_repeats=20)

imp_df = pd.DataFrame({
    "feature": feature_names,
    "perm_mse_increase": imps
}).sort_values("perm_mse_increase", ascending=False)

print(f"Baseline test MSE: {base_mse:.4f}")
display(imp_df.head(20))

# Plot top 15
top = imp_df.head(15).iloc[::-1]  # reverse for nicer barh
plt.figure(figsize=(8, 6))
plt.barh(top["feature"], top["perm_mse_increase"])
plt.xlabel("MSE increase when permuted")
plt.title("Permutation Importance (Test set) — Top 15 features")
plt.tight_layout()
plt.show()


In [ ]:
# 🔎 Combined P1–P3: Permutation importance on ICU assignment head (uses encoded test set)

import numpy as np, pandas as pd, torch
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# 0) Preconditions / guardrails
assert 'pre_torch' in globals(), "pre_torch (fitted ColumnTransformer) not found."
assert 'Xte_np' in globals() and 'yte_np' in globals(), "Use the encoded test arrays from preprocessing."
assert 'model' in globals(), "Trained model not found."

# 1) Recover feature names from fitted encoder
cat_names = pre_torch.named_transformers_['cat'].get_feature_names_out(FEATS_CAT)
num_names = np.array(FEATS_NUM, dtype=str)
feature_names = np.concatenate([cat_names, num_names])

assert feature_names.shape[0] == Xte_np.shape[1], f"Feature name count {feature_names.shape[0]} != X cols {Xte_np.shape[1]}"

# 2) Predict helper for the ICU assignment head (target column 0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.eval(); model.to(device)

def predict_assign_numpy(X_np: np.ndarray) -> np.ndarray:
    X_t = torch.tensor(X_np, dtype=torch.float32, device=device)
    with torch.no_grad():
        out = model(X_t)
        # Handle multi-task vs single-task models
        if isinstance(out, (tuple, list)):   # multi-task: (assign, bottle, ...)
            assign = out[0]
        else:                                 # single head model
            assign = out
    return assign.detach().cpu().numpy().ravel()

# 3) Baseline test MSE for assignment head
y_true = yte_np[:, 0]  # ICU assignment target
y_pred = predict_assign_numpy(Xte_np)
baseline_mse = mean_squared_error(y_true, y_pred)
print(f"Baseline test MSE (assignment): {baseline_mse:.4f}")

# 4) Permutation importance (manual, robust)
rng = np.random.default_rng(42)
n_repeats = 10
mse_increase = np.zeros(Xte_np.shape[1], dtype=float)

for j in range(Xte_np.shape[1]):
    losses = []
    for _ in range(n_repeats):
        X_perm = Xte_np.copy()
        rng.shuffle(X_perm[:, j])  # in-place column shuffle
        yp = predict_assign_numpy(X_perm)
        losses.append(mean_squared_error(y_true, yp))
    mse_increase[j] = np.mean(losses) - baseline_mse

imp_df = pd.DataFrame({"feature": feature_names, "mse_increase": mse_increase})
imp_df = imp_df.sort_values("mse_increase", ascending=False)
display(imp_df.head(20))

# 5) Grouped importance (ICU capacity vs Non-ICU capacity vs Other)
icu_mask = np.array([f.startswith("cap_") and "_icu" in f for f in feature_names])
nonicu_mask = np.array([f.startswith("cap_") and "_icu" not in f for f in feature_names])
other_mask = ~(icu_mask | nonicu_mask)

group_df = pd.DataFrame({
    "group": ["ICU capacity", "Non-ICU capacity", "Other"],
    "mean_perm_mse_increase": [
        imp_df.loc[icu_mask, "mse_increase"].mean(),
        imp_df.loc[nonicu_mask, "mse_increase"].mean(),
        imp_df.loc[other_mask, "mse_increase"].mean(),
]})
display(group_df.sort_values("mean_perm_mse_increase", ascending=False))

# 6) Quick plot of top-15 features (optional)
top = imp_df.head(15).iloc[::-1]
plt.figure(figsize=(8, 6))
plt.barh(top["feature"], top["mse_increase"])
plt.xlabel("MSE increase when permuted")
plt.title("Permutation Importance (ICU assignment head) — Top 15")
plt.tight_layout()
plt.show()
